In [34]:
import dill
import matplotlib.pyplot as plt

In [35]:
import importlib.util
import sys
import os

module_path = os.path.abspath("../1_datasets/USAVars.py")

spec = importlib.util.spec_from_file_location("USAVars", module_path)
module = importlib.util.module_from_spec(spec)
sys.modules["USAVars"] = module
spec.loader.exec_module(module)

USAVars = module.USAVars


In [36]:
train_data = USAVars(root="/share/usavars", isTrain=True)
label = 'population'

In [46]:
cost_path = f"/home/libe2152/optimizedsampling/0_data/costs/usavars/{label}/convenience_costs/capped_7_linear_distance_km_based_costs_top20_urban_0.01.pkl"

In [47]:
with open(cost_path, "rb") as f:
    arrs = dill.load(f)

In [48]:
ids = arrs['ids']
costs = arrs['costs']

In [49]:
ids_to_costs = dict(zip(ids, costs))

In [50]:
ids_to_costs

{'1535,2258': np.float64(4.3888708532826675),
 '900,3142': np.float64(2.740164861514327),
 '1851,2819': np.float64(4.749892680812925),
 '1692,3161': np.float64(3.9471669723000753),
 '1119,2918': np.float64(5.020715611512146),
 '1020,2271': np.float64(6.245802512536365),
 '620,1534': np.float64(6.05285300092515),
 '858,2628': np.float64(1.7564816341616711),
 '1082,2833': np.float64(4.078442567724997),
 '541,767': np.float64(7.0),
 '1047,521': np.float64(5.5080063396751076),
 '126,1870': np.float64(7.0),
 '1707,2425': np.float64(5.718405722282781),
 '1609,1974': np.float64(2.2201610451216345),
 '1683,1699': np.float64(5.337828392306262),
 '1637,2121': np.float64(2.230024546436255),
 '619,2894': np.float64(3.47721636777533),
 '679,2552': np.float64(3.556111971367079),
 '1755,1101': np.float64(3.618418294174673),
 '1467,3140': np.float64(3.852645886960827),
 '205,2138': np.float64(4.94962684263178),
 '2104,1979': np.float64(4.184897368968828),
 '1582,2346': np.float64(5.162021086287877),
 

In [51]:
costs = [ids_to_costs[id_] for id_ in train_data.ids]

In [52]:
import geopandas as gpd

In [53]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np

def plot(
    costs,
    country_shape_file: str = '/home/libe2152/optimizedsampling/0_data/boundaries/us/us_states_provinces/ne_110m_admin_1_states_provinces.shp',
    exclude_names: list[str] = ['Alaska', 'Hawaii', 'Puerto Rico'],
    point_size: float = 3,
    cmap: str = 'viridis',
    title: str = None,
):
    """
    Plot selected lat/lon points on a country shapefile, colored by cost.

    Args:
        costs: list or array of cost values corresponding to lat/lon points.
        country_shape_file: path to a shapefile for plotting the country boundary.
        exclude_names: list of names to exclude from the shapefile.
        point_size: size of plotted points.
        cmap: matplotlib colormap name.
        title: optional plot title.

    Returns:
        A matplotlib Figure showing the points on the map.
    """
    print("Plotting latlon subset with costs...")
    # Load country shapefile
    country = gpd.read_file(country_shape_file)
    if exclude_names:
        country = country[~country['name'].isin(exclude_names)]

    # Prepare GeoDataFrame of points
    latlons_subset = train_data.latlons  # expects np.ndarray shape (N, 2)
    assert len(costs) == len(latlons_subset), "Mismatch between costs and lat/lon points."

    points_gdf = gpd.GeoDataFrame(
        {'cost': costs},
        geometry=gpd.points_from_xy(latlons_subset[:, 1], latlons_subset[:, 0]),
        crs='EPSG:4326'
    )

    # Plot
    fig, ax = plt.subplots(figsize=(12, 10))
    country.plot(ax=ax, edgecolor='black', facecolor='none')

    # Normalize colors
    norm = mcolors.Normalize(vmin=np.min(costs), vmax=np.max(costs))
    points = points_gdf.plot(
        ax=ax,
        column='cost',
        cmap=cmap,
        markersize=point_size,
        legend=True,
        norm=norm
    )

    if title:
        ax.set_title(title, fontsize=14)

    ax.set_axis_off()
    plt.tight_layout()
    plt.show()
    return fig


In [ ]:
plot(costs)

Plotting latlon subset with costs...
